In [1]:
from src.helper_functions import hamming_distance, has_repeated_blocks
from src.symmetric_encryption import is_ecb_mode
from src.xor import xor_hex_strings
# 
import src.analysis as analysis
import src.aes128 as aes128
import src.convert as convert
import src.padding as padding
import src.load as load
import src.xor as xor
# 
import base64

0.5000028839062313
0.6699420031683737


## **Challenge 1: Convert hex to base64**

In [2]:
convert.hex2b64("49276d206b696c6c696e6720796f757220627261696e206c696b65206120706f69736f6e6f7573206d757368726f6f6d")

b'SSdtIGtpbGxpbmcgeW91ciBicmFpbiBsaWtlIGEgcG9pc29ub3VzIG11c2hyb29t'

## **Challenge 2: Fixed XOR**

In [3]:
xor_hex_strings(
    "1c0111001f010100061a024b53535009181c", 
    "686974207468652062756c6c277320657965",
).hex()

'746865206b696420646f6e277420706c6179'

## **Challenge 3: Single-byte XOR cipher**

In [4]:
cyphertext = convert.hex2bytes("1b37373331363f78151b7f2b783431333d78397828372d363c78373e783a393b3736")

best_plaintext = ""
best_score = 0
for i in range(256):
    key = convert.int2bytes(i)
    plaintext = xor.single_byte_decrypt(cyphertext, key)
    plaintext = plaintext.decode("ascii", "ignore")
    score = analysis.score_string(plaintext)
    
    # Keep track of most likely plaintext
    if score > best_score:
        best_plaintext = plaintext
        best_score = score

print(f"The best plaintext is: \n{best_plaintext}")

The best plaintext is: 
Cooking MC's like a pound of bacon


## **Challenge 4: Detect single-character XOR**

In [5]:
cyphertext_list = load.file_as_bytes("challenge_data/4.txt")
cyphertext_list = cyphertext_list.split(b"\n")

best_plaintext = ""
best_score = 0
for i in range(len(cyphertext_list)):
    cyphertext = cyphertext_list[i].decode("ascii")
    cyphertext = convert.hex2bytes(cyphertext)

    for j in range(256):
        key = convert.int2bytes(j)
        plaintext = xor.single_byte_decrypt(cyphertext, key)
        plaintext = plaintext.decode("ascii", "ignore")
        score = analysis.score_string(plaintext)
        
        # Keep track of most likely plaintext
        if score > best_score:
            best_plaintext = plaintext
            best_score = score


print(f"The best plaintext is: \n{best_plaintext}")

The best plaintext is: 
Now that the party is jumping



## **Challenge 5: Implement repeating-key XOR**

In [6]:
string = """Burning 'em, if you ain't quick and nimble
I go crazy when I hear a cymbal"""
byte_str = convert.string2bytes(string)
xor_key = convert.string2bytes("ICE")

xor.repeating_key_encrypt(byte_str, xor_key).hex()

'0b3637272a2b2e63622c2e69692a23693a2a3c6324202d623d63343c2a26226324272765272a282b2f20430a652e2c652a3124333a653e2b2027630c692b20283165286326302e27282f'

## **Challenge 6: Break repeating-key XOR**
# TODO: THIS CHALLENGE IS NOT COMPLETE

In [7]:
# Check that the hamming distance function works
a = b"this is a test"
b = b"wokka wokka!!!"

# should be 37
print(f"The hamming distance between {a} and {b} is {hamming_distance(a, b)}")

The hamming distance between b'this is a test' and b'wokka wokka!!!' is 37


In [8]:
cyphertext = load.file_as_bytes("challenge_data/6.txt", remove_newlines=True)

# Find the key_size with the minimum hamming distance
min_dist = float("inf")
min_key_size = 0
for key_size in range(1, 41):
    blocks = convert.bytes2blocks(padding.apply_pkcs_7(cyphertext, key_size), key_size)
    
    # Calculate the hamming distance between each block
    a = hamming_distance(blocks[0], blocks[1])
    b = hamming_distance(blocks[1], blocks[2])
    c = hamming_distance(blocks[2], blocks[3])
    d = hamming_distance(blocks[3], blocks[4])

    # Normalize the hamming distance
    dist = (a + b + c + d) / key_size
    print(f"Key size {key_size} has a normalized hamming distance of {dist}")
    if dist < min_dist:
        min_dist = dist
        min_key_size = key_size

print(f"The minimum hamming distance is {min_dist}, with key_size {min_key_size}")

Key size 1 has a normalized hamming distance of 15.0
Key size 2 has a normalized hamming distance of 12.5
Key size 3 has a normalized hamming distance of 10.333333333333334
Key size 4 has a normalized hamming distance of 11.0
Key size 5 has a normalized hamming distance of 13.0
Key size 6 has a normalized hamming distance of 13.166666666666666
Key size 7 has a normalized hamming distance of 12.714285714285714
Key size 8 has a normalized hamming distance of 12.25
Key size 9 has a normalized hamming distance of 12.88888888888889
Key size 10 has a normalized hamming distance of 13.7
Key size 11 has a normalized hamming distance of 12.636363636363637
Key size 12 has a normalized hamming distance of 12.25
Key size 13 has a normalized hamming distance of 12.153846153846153
Key size 14 has a normalized hamming distance of 12.714285714285714
Key size 15 has a normalized hamming distance of 12.866666666666667
Key size 16 has a normalized hamming distance of 11.8125
Key size 17 has a normalized 

In [14]:
best_plaintext = b""
best_score = 0
key_size = 10
best_key = b"\x00" * key_size

for j in range(key_size):
    for i in range(256):
        key = best_key[:j] + convert.int2bytes(i) + best_key[j+1:]
        plaintext = xor.repeating_key_decrypt(cyphertext, key)
        plaintext = plaintext.decode("ascii", "ignore")
        score = analysis.score_string(plaintext)
        
        # Keep track of most likely plaintext
        if score > best_score:
            best_plaintext = plaintext
            best_score = score
            best_key = key

    print(f"Best Key {best_key}")
    print(f"Best Score {best_score}")
    print(f"Best Plaintext {best_plaintext}")

print(f"The best plaintext is: \n{best_plaintext} with key {best_key}")

Best Key b"'\x00\x00\x00\x00\x00\x00\x00\x00\x00"
Best Score 0.6908247602798366
Best Plaintext oUIfTQsPAhPE048Gmllo0kcDk4TAQTHThsBFkU2fB4BSWQgVBdQzNTTmVSegBHVBwNRUHBAxTEjwMoghJGgkRTxuMIRpHKwAFoUdZEQQJAGvmB1MANxYGcBoXQR0BUlvwXwAgEwoFu08SSAhFTmr+Fgk4RQYFdBpGB08fWXO+amI2DB0PvQ1IBlUaGwfdQnQEHgFJ`gkRAlJ6f0LASDoAGhNJ`k9FSA8dDVjEOgFSGQELvRMGAEwxX1iiFQYHCQdUdxdBFBZJeTj1CxsBBQ9Ge08dTnhOSCCSBAcMRVhIdEEATyBUCHvLHRlJAgAOalwAUjBpZRJAgJUAAELe04CEFMBJhfVTQIHAh9P`054MGk2UgHBCVQGBwlTsgIQUwg7EA~FSQ8PEE87fDpfRyscSWvzT1QCEFMaswUWEXQMBkPAg4DQ1JMwU4ALwtJDQOOFw0VVB1PchxFXigLTRLBEgcKVVN4sk9iBgELR1jdDAAAFwoFoww6Ql5NLgaBIg4cSTRWvWI1Bk9HKn7CE8BGwFTvjcEBx4MThrcDgYHKxpUlhdJGQZZVCaFVwcDBVMHjUV4LAcKQRJUlk3TwAmoQdJEwATARiFTg5JFwQ5d15NHQYEGk4dzBDADsdoE4UVBUaDEJTwgHRTkArmc6AUETCg~AN1xGYlUKcxJTEUgsAAABwcXOwlS`QELQQcbE0D9GioWGgwcfgcHSAtPTgTAABY9C1VNdAINGxgXRH@waWUfSQcJfBkRRU8ZAULDDTUWF01jhgkRTxVJKl}JJwFJHQYAcUgRSAsWSRKIgBSAAxOfBoLUlQwW1uiGxpOCEtU~iROCk8gUw~1C1IJCAACbU8QRSxORTeSHQYGTlQJd1lOBAAXRTWCUh0FDxhU}XhzL

## **Challenge 7: AES in ECB mode**

In [10]:
data = load.file_as_bytes("challenge_data/7.txt", remove_newlines=True)
data = base64.decodebytes(data)
decrypted_output = aes128.ecb_decrypt(data, b"YELLOW SUBMARINE")

# Display the decrypted output
print(decrypted_output.decode("ascii", errors="ignore"))

I'm back and I'm ringin' the bell 
A rockin' on the mike while the fly girls yell 
In ecstasy in the back of me 
Well that's my DJ Deshay cuttin' all them Z's 
Hittin' hard and the girlies goin' crazy 
Vanilla's on the mike, man I'm not lazy. 

I'm lettin' my drug kick in 
It controls my mouth and I begin 
To just let it flow, let my concepts go 
My posse's to the side yellin', Go Vanilla Go! 

Smooth 'cause that's the way I will be 
And if you don't give a damn, then 
Why you starin' at me 
So get off 'cause I control the stage 
There's no dissin' allowed 
I'm in my own phase 
The girlies sa y they love me and that is ok 
And I can dance better than any kid n' play 

Stage 2 -- Yea the one ya' wanna listen to 
It's off my head so let the beat play through 
So I can funk it up and make it sound good 
1-2-3 Yo -- Knock on some wood 
For good luck, I like my rhymes atrocious 
Supercalafragilisticexpialidocious 
I'm an effect and that you can bet 
I can take a fly girl and make her wet. 


## **Challenge 8: Detect AES in ECB mode**

In [11]:
with open("challenge_data/8.txt", "r") as f:
    lines = f.readlines()
    for i in range(len(lines)):
        hex_str = lines[i].replace("\n", "")
        hex_bytes = convert.hex2bytes(hex_str)
        if has_repeated_blocks(hex_bytes, 16):
            print(f"ECB mode detected in line {i}!")
            break

ECB mode detected in line 132!
